# Small Multiples

Small multiples are important for visualizing higher dimensional data. The key idea is to slice up your data and use a separate plot to visualize each slice. The end result is a grid of charts that all follow the same visual format, but show different pieces of the data.

From <a href="http://en.wikipedia.org/wiki/Small_multiple" target="_blank">Wikipedia</a>:

>A small multiple (sometimes called trellis chart, lattice chart, grid chart, or panel chart) is a series or grid of small similar graphics or charts, allowing them to be easily compared. The term was popularized by Edward Tufte. 

Essentially, a chorus of little stories to help tell a bigger one.

While the concept is simple, the benefits can be significant. Compared to a single larger chart, small multiples can help with overplotting — when data is obscured or occluded because there are too many plotted items.

Compared to animation, small multiples present all of the data at once making it easier for viewers to naturally compare each facet with others, instead of trying to flip back and forth between views.

Viewers can quickly learn to read an individual plot and apply this knowledge as they scan the rest of the visualization. This shifts the reader’s effort from understanding *how the visualization works* to *what the data says*. 


## Goals

In this Jupyter Notebook, we'll look at a few ways to create small multiples visualizations in Python. Specifically, we'll look at:

* matplotlib
* Seaborn

Both of them have strengths and weaknesses, and either of them can help us visually exploring data.

### If you haven't already installed Seaborn, use `conda install seaborn`.

## matplotlib

We'll start with the most familiar and fundamental Python plotting library. As you'll see, `matplotlib` allows a lot of flexibility to create small multiples visualizations. You'll also see that it requires a lot of "hands on" coding.

Rather than download and clean and wrangle data for this example, we'll just start with a dataset taken from [Ask MetaFilter](http://stuff.metafilter.com/infodump/) that is ready to use.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
# for this example, we'll use a matplotlib function that lets us automate some things
import matplotlib.ticker as mticker

In [ ]:
! head askmefi_category_year.tsv

In [ ]:
data = pd.read_csv('askmefi_category_year.tsv', sep='\t')

In [ ]:
data.head()

In [ ]:
data.tail()

This dataset shows the number of user posts about each of a variety of categories for about 10 years. We will use this data to create small multiples of posts over time. Each plot will display a different category of posts.

In [ ]:
data.category.unique()

We're going to use a dictionary to rearrange the data for plotting. We could take a different approach using `groupby`, but let's stick with this for now.

In [ ]:
categories = data.category.unique()
mydata = {}
for i in categories:
    mydata[i] = data[data.category==i]

In [ ]:
len(mydata)

In [ ]:
mydata

In [ ]:
mydata[categories[0]].head()

Here's the plotting code in one large block. I've added some comments to help explain things.

In [ ]:
nrows = 4; ncols = 5
num_plots = nrows * ncols  # number of subplots
assert num_plots == len(mydata) # checks the math

figwidth = 13/1.1
figheight = 10/1.2

x_range = range(2004,2015,1)

fig = plt.figure(figsize=(figwidth, figheight))

# create the subplot figure
axes = [plt.subplot(nrows,ncols,i) for i in range(1,num_plots+1)]

# h_pad is horizontal padding between subplots
# w_pad is vertical padding between subplots
plt.tight_layout(pad=0, w_pad=3, h_pad=3)
plt.subplots_adjust(hspace=1)
plt.rcParams['xtick.major.pad']='8'

for i in range(len(categories)):
    ax = axes[i]
    y = data["n"][data.category==categories[i]]
    x = x_range
    ax.plot(x,y,color='#222222')
    ax.fill_between(x,y,0,color='#cec6b9')

    #ax.xticks(['2004','2014'])
    ax.set_ylim([0,3000])
    ax.set_xlim([2004,2014])
    
    # Remove top and right axes and ticks
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)

    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #ax.xticks([0,1], ['2004','2014'], rotation='horizontal')    
    # Find at most 101 ticks on the y-axis at 'nice' locations
    max_yticks = 3
    yloc = plt.MaxNLocator(max_yticks)
    ax.yaxis.set_major_locator(yloc)

    #max_xticks = 4
    #xloc = plt.MaxNLocator(max_xticks)
    #ax.xaxis.set_major_locator(xloc)
    ax.set_xticklabels(['2004','','','','','2014'])
    
    ax.set_xlabel(categories[i])
    ax.tick_params(axis='both', which='major', labelsize=8)
    
    def func(x, pos):  # formatter function takes tick label and tick position
       s = '{:0,d}'.format(int(x))
       return s

    ax.yaxis.set_major_formatter(mticker.FuncFormatter(func)) 

plt.show()

### That took quite a bit of effort!

## Seaborn

Those of us familiar with using R for data analytics depend on a couple of packages (libraries) for making better plots and doing it much easier. One R package is `Lattice`. The other is `ggplot2`. Fortunately, there are Python libraries that provide similar (easier) functionality. We'll look at the `Seaborn` library.

For this example, we'll use a "classic" sample dataset with information about tipping in restaurants. This dataset in included in Seaborn, so we will simply load it.

## Use example from here: https://seaborn.pydata.org/generated/seaborn.FacetGrid.html

In [ ]:
import seaborn as sns; sns.set(style="ticks", color_codes=True)

In [ ]:
tips = sns.load_dataset("tips")
#g = sns.FacetGrid(tips, col="time", row="smoker")

Here's how plotting works with `Seaborn`. First we create a plot object. Then we add layers to it.

We'll use an example from the [Seaborn documentation](https://seaborn.pydata.org/generated/seaborn.FacetGrid.html).

Draw a univariate plot on each facet.

In [ ]:
import matplotlib.pyplot as plt
g = sns.FacetGrid(tips, col="time",  row="smoker")
g.map(plt.hist, "total_bill");

That was easy!

We can pass additional keyword arguments to the mapped function:

In [ ]:
import numpy as np
bins = np.arange(0, 65, 5)
g = sns.FacetGrid(tips, col="time",  row="smoker")
g.map(plt.hist, "total_bill", bins=bins, color="r");

We can change the plot type just by changing the "geom" name. 

Let's plot scatter plots on each facet.

Note that some of the terminology here comes from the [Grammar of Graphics](http://en.wikipedia.org/wiki/Leland_Wilkinson#Professional_career).

In [ ]:
g = sns.FacetGrid(tips, col="time",  row="smoker")
g.map(plt.scatter, "total_bill", "tip", edgecolor="w");

Assign one of the variables to the color of the plot elements:

In [ ]:
g = sns.FacetGrid(tips, col="time",  hue="smoker")
(g.map(plt.scatter, "total_bill", "tip", edgecolor="w").add_legend());

Change the height and aspect ratio of each facet:

In [ ]:
g = sns.FacetGrid(tips, col="day", height=4, aspect=.5)
g.map(plt.hist, "total_bill", bins=bins);

Specify the order for plot elements:

In [ ]:
g = sns.FacetGrid(tips, col="smoker", col_order=["Yes", "No"])
g.map(plt.hist, "total_bill", bins=bins, color="m");

Use a different color palette:

In [ ]:
kws = dict(s=50, linewidth=.5, edgecolor="w")
g = sns.FacetGrid(tips, col="sex", hue="time", palette="Set1", hue_order=["Dinner", "Lunch"])
(g.map(plt.scatter, "total_bill", "tip", **kws).add_legend());

Use a dictionary mapping hue levels to colors:

In [ ]:
pal = dict(Lunch="seagreen", Dinner="gray")
g = sns.FacetGrid(tips, col="sex", hue="time", palette=pal, hue_order=["Dinner", "Lunch"])
(g.map(plt.scatter, "total_bill", "tip", **kws).add_legend())

Additionally use a different marker for the hue levels:

In [ ]:
g = sns.FacetGrid(tips, col="sex", hue="time", palette=pal, hue_order=["Dinner", "Lunch"], hue_kws=dict(marker=["^", "v"]))
(g.map(plt.scatter, "total_bill", "tip", **kws).add_legend())

Now we'll load another dataset and “wrap” a column variable with many levels into the rows:

In [ ]:
att = sns.load_dataset("attention")
g = sns.FacetGrid(att, col="subject", col_wrap=5, height=1.5)
g.map(plt.plot, "solutions", "score", marker=".");

The Seaborn tutorial continues with more elaborate examples, but I think we'll all agree that using Seaborn is much simpler than using matplotlib for these lattice/facet/small multiples visualizations.

If you are interested in learning how to use Tableau to create small multiples visualizations, I recommend that you go here: https://www.ryansleeper.com/how-to-make-trellis-tile-small-multiple-maps-in-tableau/